# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
***

## Import Modules <a name="import-modules"></a>

In [1]:
# iPython Settings
%load_ext autoreload
%autoreload 2

# Python Modules
import sys
sys.path.insert(0, "..")

import pandas as pd
pd.options.mode.chained_assignment = None
from ase.visualize import view
import plotly.graph_objs as go
import plotly.plotly as py

# My Modules ******************************************************************
# from orr_reaction.orr_methods import lowest_e_path, df_calc_adsorption_e, plot_all_states
from orr_reaction.orr_methods import ORR_Free_E_Plot
from orr_reaction.orr_methods import lowest_e_path

from plotting.my_plotly import reapply_colors
from colors.colors import color_list

# Local Imports
from an_data_processing import load_df

ImportError: cannot import name 'ORR_Free_E_Plot'

### Script Inputs

In [ ]:
bias = 0.


# Smart Formatting
smart_format = [

    # [
    #     {"spinpol": True},
    #     {"dash": "dot"},
    #     ],

    [
        {"system": "Fe_slab"},
        {"dash": "dashdot"},
        ],

    [
        {"system": "N_graph_Fe"},
        {"dash": "dot"},
        ],

    [
        {"system": "graph_Fe"},
        {"dash": "dash"},
        ],

    [
        {"system": "graphene"},
        {"dash": None},
        ],

    ]


In [ ]:
df = load_df(from_file=True, root_dir="..")

In [ ]:
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase import io
# io.write("out.traj", df.loc[2].atoms_object)
df_tmp = df[df["system"] == "N_graph_Fe"]

images = df_tmp.iloc[0].atoms_object
atoms = images[0]

# io.write("out.traj", atoms)
# view(images)

atoms_i = df.iloc[0].atoms_object[-1]
# view(atoms_i)

# Free Energy Diagram <a name="fed_all"></a>

In [ ]:
dat_lst_master = []

for i_ind, group_i in enumerate(df.groupby("system")):
    df_i = group_i[1]
    
    # Tree Level Labels for group_i
    tree_labels = df_i["tree_level_labels"].tolist()
    tree_labels_2 = []
    for item in tree_labels:
        if sorted(item) not in tree_labels_2:
            tree_labels_2.append(sorted(item))
    assert len(tree_labels_2) == 1
    tree_labels_2 = tree_labels_2[0]

    # i_ind = i_ind * 2
    color_list_i = color_list[i_ind:i_ind + 2]

#     print(group_i[0])
    
    dat_lst, layout = lowest_e_path(
        df_i,
        tree_labels_2,
        color_list_i,
        create_ideal_series=False,
        opt_name=df_i["system"].iloc[0],
        bias=bias,
        manual_props="False",
#         smart_format=smart_format,
        smart_format=None,
        )
    dat_lst_master += dat_lst  

# Adding Ideal ORR line
ORR = ORR_Free_E_Plot()
e_list_ideal = ORR.apply_bias(bias, ORR.ideal_energy)
dat_ideal = ORR.create_plotly_series(
    e_list_ideal,
    group="Ideal",
    name="Ideal",
    color="red",
    plot_mode="full_lines",
    )

dat_lst_master += dat_ideal

dat_lst_master = reapply_colors(dat_lst_master)

In [ ]:
py.iplot(
    {
        "data": dat_lst_master,
        "layout": layout,
        },
    filename="01_fe_graph/pl_fed_all_systems.html"
    )